Format of JSON Responses
JSON responses are specific to the API call and are as lightweight as possible. If the JSON object includes a list of objects, such as artists, then the key will be the ID for that object (for example, the artist-id).
Also, note that timestamps in the JSON responses are day counts since the Unix epoch. To translate these to standard Unix timestamps in seconds, multiply the day count by 86,400.
(replace 'xml' with 'json' in 

View
Retrieve the name and MusicBrainzId for an artist, given an NBS Artist ID.
http://key.api3.nextbigsound.com/artists/view/ARTIST-ID.xml JSON
Search
Search the NBS artist database for an artist.
http://key.api3.nextbigsound.com/artists/search.xml?q=QUERYJSON
Add requires a private key
Add an artist to the NBS database.
http://key.api3.nextbigsound.com/artists/add.xml JSON
This call requires the following POST variables
key: your private key
name: name of the artist
profiles: an array of profile urls. At least one is required to successfully add the artist
GENRES
Artist
Retrieve the genres for an NBS artist.
http://key.api3.nextbigsound.com/genres/artist/ARTIST-ID.xml JSON
METRICS
The following is a list of the variables that can optionally be POST-ed to the url endpoint.
data[start]: a unix timestamp or a text-based date [see php.net/strtotime for details]
data[end]: a unix timestamp or a text-based date
data[metric]: all, plays, fans, views, comments, downloads, likes, price
Visit our API testing tool for more information on the two metric feeds. This tool will provide insight on how to specify a timespan and metric type.
Profile
Retrieve social media data for a NBS profile such as a MySpace or Facebook page.
http://key.api3.nextbigsound.com/metrics/profile/PROFILE-ID.xml JSON
Artist
Retrieve social media data for a NBS artist, across social media sites.
http://key.api3.nextbigsound.com/metrics/artist/ARTIST-ID.xml JSON
PROFILES
Artist
Retrieve the social media profiles for an NBS artist.
http://key.api3.nextbigsound.com/profiles/artist/ARTIST-ID.xml JSON
Search
Search our social media profile database for a given social media url. Artists associated to that url are returned.
http://key.api3.nextbigsound.com/profiles/search.xml?u=URL JSON
Add requires a private key
Add a social media profile to an NBS artist.
http://key.api3.nextbigsound.com/profiles/add/ARTIST-ID.xml JSON
This call requires the following POST variables
key: your private key
profiles: an array of profile urls.
SERVICES
List
Retrieve all the social media sites that we support.
http://key.api3.nextbigsound.com/services.xml JSON


In [2]:
import anyjson as json
import requests
from bs4 import BeautifulSoup
#import scalpyr
import urllib
from json import load 
import pandas as pd
#from pandas import DataFrame
import numpy as np
import codecs
#from urllib import request
from urllib2 import urlopen # python 2.7 version
#import urlopen
from json import load
#import sqlite3
#import csv
#import codecs
#import cStringIO
#import sys
import hashlib
import datetime as dt

In [3]:
# nbs api python wrapper
from nbs_api import API
import json

#api = API("nbsmobile")
api = API("swahl")

In [6]:
# search artist
resp = json.loads(api.artistSearch("Radiohead"))
#print resp
filtresp = {key:resp[key] for key in resp.keys() if (resp[key]['music_brainz_id'] is not None) }
print filtresp

{u'797': {u'name': u'Radiohead', u'music_brainz_id': u'a74b1b7f-71a5-4011-9441-d0b5e4122711'}}


In [62]:
# basic musicbrainz search 
import musicbrainzngs
import sys

musicbrainzngs.set_useragent(
    "cdips-music-data-project",
    "0.1",
    'https://github.com/trxw/CDIPS_PandoraTeam',
)

artist_id = filtresp['797']['music_brainz_id']
try:
    result = musicbrainzngs.get_artist_by_id(artist_id)
except WebServiceError as exc:
    print("Something went wrong with the request: %s" % exc)
else:
    artist = result["artist"]
    print("name:\t\t%s" % artist["name"])
    print("sort name:\t%s" % artist["sort-name"])

name:		Radiohead
sort name:	Radiohead


In [7]:
# services summary
resp = json.loads(api.servicesList())
nbs_services = {key:resp[key]['human'] for key in resp.keys()}
nbs_services

{u'1': u'MySpace',
 u'10': u'SoundCloud',
 u'11': u'Purevolume',
 u'16': u'Vimeo',
 u'17': u'Wikipedia',
 u'2': u'Last.fm',
 u'25': u'Vevo',
 u'33': u'Rdio',
 u'4': u'Facebook',
 u'45': u'Bandcamp',
 u'5': u'Twitter',
 u'52': u'Instagram',
 u'64': u'Tumblr',
 u'66': u'Goodreads (Author)',
 u'68': u'Goodreads (Work)',
 u'7': u'YouTube',
 u'70': u'NBS Score',
 u'77': u'Vine',
 u'8': u'ReverbNation',
 u'85': u'GooglePlus'}

In [9]:
import re
def cleanName(name):
    s = name
    if ', The' in s:
        s = 'The ' + s[:-5]
    r = re.compile(r"-")
    s = r.sub(" ", s) 
    #r = re.compile(r"[+&]")
    #fix_bad[i] = r.sub("and", fix_bad[i]) 
    return s

cleanName('Jay-Z, The')

'The Jay Z'

In [14]:
# use artists from
artist_names = pd.DataFrame.from_csv('../billboard/billboard_artist_names_2010_15.csv')
print len(artist_names)
artist_names.head()
artist_names['artist_name_clean'] = artist_names.artist_name.apply(cleanName)
artist_names.head()

631


,artist_name,artist_name_clean
0,Ke$ha,Ke$ha
1,Eminem,Eminem
2,Katy Perry,Katy Perry
3,Rihanna,Rihanna
4,Bruno Mars,Bruno Mars


In [16]:
# combined nbs + musicbrainz aritist table
# Currently losing about half of the queries to key errors and non-unique results
# recognized key errors:
# "*, the'
# "*-*"
# "*+*"

from nbs_api import API
import musicbrainzngs
import sys
#api = API("nbsmobile")

#queries = ['Jay Z', 'The Black Keys','Selena Gomez','Florence and the Machine', 'Maddie and Tae']
queries = artist_names.artist_name_clean.unique()
#queries = fix_bad
bad_queries = []

musicbrainzngs.set_useragent(
    "cdips-music-data-project",
    "0.1",
    'https://github.com/trxw/CDIPS_PandoraTeam',
)

# dataframe
artists = pd.DataFrame()

for query in queries:
    # search artist
    try:
        resp = json.loads(api.artistSearch(query))
        #print resp
        filtresp = {key:resp[key] for key in resp.keys() if (resp[key]['music_brainz_id'] is not None) }
        # if multiple responses see if exact match
        if len(filtresp) > 1:
            try:
                for key in filtresp.keys():
                    if filtresp[key]['name']==query:
                        filtresp = {key:filtresp[key]}
            except:
                pass
        print filtresp
        
        if len(filtresp) == 1:
            for key in filtresp.keys():
                nbs_id = key
                mbz_id = filtresp[key]['music_brainz_id']
                nbs_name = filtresp[key]['name']

                try:
                    result = musicbrainzngs.get_artist_by_id(mbz_id)
                except WebServiceError as exc:
                    print("Something went wrong with the request: %s" % exc)
                else:
                    mbz_name = ''; mbz_name=''; country='';begin_area=''
                    life_span_begin='';mbz_type='';isni_list=''
                    try: 
                        mbz_name = result['artist']['name']
                    except:
                        pass
                    try:
                        area = result['artist']['area']['name']
                    except:
                        pass
                    try:
                        country = result['artist']['country']
                    except:
                        pass                    
                    try:
                        begin_area = result['artist']['begin-area']['name']
                    except:
                        pass                    
                    try:
                        life_span_begin = result['artist']['life-span']['begin']
                    except:
                        pass                    
                    try:
                        mbz_type = result['artist']['type']
                    except:
                        pass                    
                    try:
                        isni_list = result['artist']['isni-list'][0]
                    except:
                        pass  

                # genres
                genres = json.loads(api.genresArtist(nbs_id))

                gencols = []
                genlist = []
                for i in (np.arange(5)+1):
                    gencols += ['genre'+str(i)]
                    try:
                        genlist += [genres.pop()['name']]
                    except:
                        genlist += ['']

                acols = ['id','name','music_brainz_id','music_brainz_name',
                        'area','country','begin_area','life_span_begin',
                        'music_brainz_type','isni_list']+gencols
                art = pd.DataFrame([[nbs_id,nbs_name,mbz_id,mbz_name,
                                    area,country,begin_area,life_span_begin,
                                   mbz_type,isni_list]+genlist],columns=acols,index=[nbs_id])
                artists = artists.append(art)
        else:
            print "No unique result for:", query
            bad_queries.append(query)
                
    except:
        print "Error for query:", query
        bad_queries.append(query)
    else:
        if len(filtresp) == 0:
            print "No result for query:", query
            bad_queries.append(query)

# convert dates
artists.life_span_begin = pd.to_datetime(artists.life_span_begin)
artists.head()

{u'269811': {u'name': u'Ke$ha', u'music_brainz_id': u'fab34286-b8e1-4879-bce3-194e1358fbd2'}}
{u'3886': {u'name': u'Eminem', u'music_brainz_id': u'b95ce3ff-3d05-4e87-9e01-c97b66af13d4'}}
{u'652': {u'name': u'Katy Perry', u'music_brainz_id': u'122d63fc-8671-43e4-9752-34e846d62a9c'}}
{u'2104': {u'name': u'Rihanna', u'music_brainz_id': u'73e5e69d-3554-40d8-8516-00cb38737a1c'}}
{u'286413': {u'name': u'Bruno Mars', u'music_brainz_id': u'afb680f2-b6eb-4cd7-a70b-a63b25c763d5'}}
{u'10709': {u'name': u'Usher', u'music_brainz_id': u'3414d446-735a-443c-931f-10634f57e5b9'}}
{u'292464': {u'name': u'Far East Movement', u'music_brainz_id': u'6bd5655c-b7fe-48ee-8925-c2d2d8798618'}}
{u'113780': {u'name': u'B.o.B', u'music_brainz_id': u'94c338ff-1985-4429-9dc8-997b61bb5932'}}
{}
No unique result for: The Black Eyed Peas
No result for query: The Black Eyed Peas
{u'9871': {u'name': u'Taio Cruz', u'music_brainz_id': u'ba7d2626-38ce-4859-8495-bdb5732715c4'}}
{u'4018': {u'name': u"Ariel Pink's Haunted Graffi

,id,name,music_brainz_id,music_brainz_name,area,country,begin_area,life_span_begin,music_brainz_type,isni_list,genre1,genre2,genre3,genre4,genre5
269811,269811,Ke$ha,fab34286-b8e1-4879-bce3-194e1358fbd2,Ke$ha,United States,US,Los Angeles,1987-03-01,Person,0000000114760658,Pop,,,,
3886,3886,Eminem,b95ce3ff-3d05-4e87-9e01-c97b66af13d4,Eminem,Detroit,,St. Joseph,1972-10-17,Person,0000000117699370,Rap,HipHop,,,
652,652,Katy Perry,122d63fc-8671-43e4-9752-34e846d62a9c,Katy Perry,United States,US,Santa Barbara,1984-10-25,Person,000000007371737X,Pop,Rock,,,
2104,2104,Rihanna,73e5e69d-3554-40d8-8516-00cb38737a1c,Rihanna,New York,,Saint Michael,1988-02-20,Person,0000000114766419,Reggae,RB,Pop,HipHop,
286413,286413,Bruno Mars,afb680f2-b6eb-4cd7-a70b-a63b25c763d5,Bruno Mars,United States,US,Honolulu,1985-10-08,Person,,Pop,Rock,,,


In [17]:
print len(np.unique(bad_queries))
print len(artists)
print (1.0*len(np.unique(bad_queries))) / len(artist_names)


106
525
0.167987321712


In [18]:
saved_bad = np.unique(bad_queries)
saved_bad

array(['A Thousand Horses', 'A Trak + Milo & Otis', 'A$AP Ferg',
       'Alison Gold', 'Amanda Brown', 'Amber Carrington', 'Anne Hathaway',
       'Antoine Dodson', 'Artists For Haiti', 'Auburn', 'Band Aid 30',
       'Blink 182', 'Bobby Shmurda', 'Brenda Lee', 'Buck 22',
       'Casey Weston', 'Cee Lo Green', 'Chase Bryant',
       'China Anne McClain', 'Chris Jamison', 'Christopher Wilde',
       'Clean Bandit', 'Craig Wayne Boyd', 'Damien', 'DeJ Loaf', 'Diggy',
       "Disney's Friends For Change", 'Driicky Graham',
       'Edward Maya & Vika Jigulina', 'Empire Cast', 'Fetty Wap',
       'Fitz And The Tantrums', 'Florence + The Machine', 'Fun.', 'Future',
       'G Eazy', 'Garrett Hedlund', 'General Larry Platt', 'Glee Cast',
       'Green Shoe Studio', 'Hustle Gang', 'I LOVE MAKONNEN', 'I Prevail',
       'Jack & Jack', 'Jacquie Lee', 'Jake Worthington', 'James Wolpert',
       'Jessie J', 'JoJo', 'Jonn Hart', 'Josh Kaufman', 'KCamp', 'Kiesza',
       'Lillywood', 'Lindsey Pavao', 

In [57]:
import re
fix_bad = np.unique(saved_bad)
# try fixing bad queries
for i,a in enumerate(np.unique(saved_bad)):
    if ', The' in a:
        fix_bad[i] = 'The ' + a[:-5]
    r = re.compile(r"-")
    fix_bad[i] = r.sub(" ", fix_bad[i]) 
    r = re.compile(r"[+&]")
    fix_bad[i] = r.sub("and", fix_bad[i]) 

fix_bad    
    

array(['A Thousand Horses', 'A Trak and Milo and Otis', 'Band Aid 30',
       'Bobby Shmurda', 'Brenda Lee', 'Buck 22', 'Chase Bryant',
       'Chris Jamison', 'Clean Bandit', 'Craig Wayne Boyd', 'Damien',
       'DeJ Loaf', 'Empire Cast', 'Fetty Wap', 'Fitz And The Tantrums',
       'Future', 'I LOVE MAKONNEN', 'I Prevail', 'Jack and Jack',
       'Jake Worthington', 'Jessie J', 'Josh Kaufman', 'KCamp', 'Kiesza',
       'Lillywood', 'LunchMoney Lewis', 'MAGIC!', 'Maddie and Tae',
       'Matt McAndrew', 'Meghan Trainor', 'Natalie La Rose',
       'O.T. Genasis', 'OG Maco', 'Rae Sremmurd', 'Rich Gang', 'Rixton',
       'Sam Hunt', 'Shawn Mendes', 'Sheppard', 'Silento', 'Snootie Wild',
       'Soko', 'Sophia Grace', 'Taylor John Williams', 'The Chainsmokers',
       'Ty Dolla $ign', 'White Buffalo and The Forest Rangers', 'Wisin',
       'Young Money'], 
      dtype='|S36')

In [50]:
#fixed_artists = artists


In [30]:
# convert datetimes in existing tables
#artists = pd.DataFrame.from_csv('tables/artists_from_billboard_2015.csv',index_col='id')
#artists.life_span_begin = pd.to_datetime(artists.life_span_begin)
#artists.to_csv('tables/artists_from_billboard_2015.csv',index_label='id',encoding="utf-8")

#artists = pd.DataFrame.from_csv('tables/artists_from_billboard_2014_15.csv',index_col='id')
#artists.life_span_begin = pd.to_datetime(artists.life_span_begin)
#artists.to_csv('tables/artists_from_billboard_2014_15.csv',index_label='id',encoding="utf-8")

In [32]:
# save table in csv file
#artists.to_csv('artists_from_billboard_2015.csv',index=False,encoding="utf-8")
#artists.to_csv('tables/artists_from_billboard_2010_15.csv',index=False,encoding="utf-8")


# load table from previous file
#artists = pd.DataFrame.from_csv('artists_from_billboard_2015.csv',index_col='id')
artists = pd.DataFrame.from_csv('tables/artists_from_billboard_2010_15.csv',index_col='id')

# append fixed artists
#artists = pd.DataFrame.from_csv('tables/artists_from_billboard_2014_15.csv',index_col='id')
#artists.append(fixed_artists)
#artists.to_csv('tables/artists_from_billboard_2014_15.csv',index=False,encoding="utf-8")

artists = artists.fillna('')
print len(artists)
artists.head()

525


,name,music_brainz_id,music_brainz_name,area,country,begin_area,life_span_begin,music_brainz_type,isni_list,genre1,genre2,genre3,genre4,genre5
id,,,,,,,,,,,,,,
269811,Ke$ha,fab34286-b8e1-4879-bce3-194e1358fbd2,Ke$ha,United States,US,Los Angeles,1987-03-01 00:00:00,Person,0000000114760658,Pop,,,,
3886,Eminem,b95ce3ff-3d05-4e87-9e01-c97b66af13d4,Eminem,Detroit,,St. Joseph,1972-10-17 00:00:00,Person,0000000117699370,Rap,HipHop,,,
652,Katy Perry,122d63fc-8671-43e4-9752-34e846d62a9c,Katy Perry,United States,US,Santa Barbara,1984-10-25 00:00:00,Person,000000007371737X,Pop,Rock,,,
2104,Rihanna,73e5e69d-3554-40d8-8516-00cb38737a1c,Rihanna,New York,,Saint Michael,1988-02-20 00:00:00,Person,0000000114766419,Reggae,RB,Pop,HipHop,
286413,Bruno Mars,afb680f2-b6eb-4cd7-a70b-a63b25c763d5,Bruno Mars,United States,US,Honolulu,1985-10-08 00:00:00,Person,,Pop,Rock,,,


In [33]:
# Create a dictionary for 
genre_group = artists.groupby('genre1').groups

for i in (np.arange(4)+2):
    col = artists['genre'+str(i)]
    for i in range(len(col)):
        try:
            genre_group[col.iloc[i]].append(col.index[i])
        except:
            genre_group.update({col.iloc[i]:[col.index[i]]})


#genre_group.keys()
genre_group.pop('')
print len(genre_group)

# print genre distribution
for key in genre_group.keys():
    print key, len(genre_group[key])

21
Electronica 16
Reggae 14
Latin 10
Christian 3
Jazz 12
Techno 5
RB 101
Hardcore 4
Progressive 1
Country 91
Metal 8
Punk 5
HipHop 133
Rock 111
Blues 11
Americana 1
Indie 22
Pop 171
Rap 58
Alternative 40
Folk 13


In [36]:
# Example: make a table of folk artists
gen = 'Blues'
print genre_group[gen]
folk_artists = artists[artists.index.isin(genre_group[gen])]
folk_artists

[2313, 13075, 509, 35697, 15281, 269934, 5478, 52266, 32944, 107631, 321669]


,name,music_brainz_id,music_brainz_name,area,country,begin_area,life_span_begin,music_brainz_type,isni_list,genre1,genre2,genre3,genre4,genre5
id,,,,,,,,,,,,,,
2313,Orianthi,15674cab-ca9c-4704-b15f-c601d2f4409b,Orianthi,Australia,AU,,1985-01-22 00:00:00,Person,0000000108023408,Blues,Country,Rock,,
13075,Alicia Keys,8ef1df30-ae4f-4dbd-9351-1a32b208a01e,Alicia Keys,United States,US,Manhattan,1981-01-25 00:00:00,Person,0000000110657805,Blues,Jazz,RB,Pop,HipHop
509,John Mayer,144ef525-85e9-40c3-8335-02c32d0861f3,John Mayer,United States,US,Bridgeport,1977-10-16 00:00:00,Person,0000000114467902,Blues,Pop,Rock,,
35697,Crystal Bowersox,23644c2d-7351-4690-81c0-0153e4652847,Crystal Bowersox,United States,US,Ohio,1985-08-04 00:00:00,Person,,Blues,Country,Rock,,
15281,Adele,cc2c9c3c-b7bc-4b8b-84d8-4fbd8779e493,Adele,United Kingdom,GB,Tottenham,1988-05-05 00:00:00,Person,0000000123144259,Blues,Pop,,,
269934,Steven Tyler,7d3f7aed-fb9c-4e80-b198-64cd6ae31cb8,Steven Tyler,United States,US,Yonkers,1948-03-26 00:00:00,Person,0000000078405834,Blues,Rock,,,
5478,The Black Keys,d15721d8-56b4-453d-b506-fc915b14cba2,The Black Keys,United States,US,Akron,2001-01-01 00:00:00,Group,,Blues,Alternative,Rock,,
52266,Vicci Martinez,6aabc385-f85b-4c98-8f68-684891a44c81,Vicci Martinez,United States,US,,1984-09-21 00:00:00,Person,,Blues,Pop,,,
32944,Hunter Hayes,9a36f5b5-9a70-4890-adfd-bc39386d354d,Hunter Hayes,United States,US,Breaux Bridge,1991-09-09 00:00:00,Person,,Blues,Country,,,


In [37]:
# Generate and print tables by genres.
for gen in genre_group.keys():
    if len(genre_group[gen]) >= 5:
        gen_artists = artists[artists.index.isin(genre_group[gen])]
        fname = 'tables/'+gen+'_artists_from_billboard_2010_15.csv'
        print fname
        gen_artists.to_csv(fname,index_label='id',encoding="utf-8")
        

tables/Electronica_artists_from_billboard_2010_15.csv
tables/Reggae_artists_from_billboard_2010_15.csv
tables/Latin_artists_from_billboard_2010_15.csv
tables/Jazz_artists_from_billboard_2010_15.csv
tables/Techno_artists_from_billboard_2010_15.csv
tables/RB_artists_from_billboard_2010_15.csv
tables/Country_artists_from_billboard_2010_15.csv
tables/Metal_artists_from_billboard_2010_15.csv
tables/Punk_artists_from_billboard_2010_15.csv
tables/HipHop_artists_from_billboard_2010_15.csv
tables/Rock_artists_from_billboard_2010_15.csv
tables/Blues_artists_from_billboard_2010_15.csv
tables/Indie_artists_from_billboard_2010_15.csv
tables/Pop_artists_from_billboard_2010_15.csv
tables/Rap_artists_from_billboard_2010_15.csv
tables/Alternative_artists_from_billboard_2010_15.csv
tables/Folk_artists_from_billboard_2010_15.csv


In [26]:
# Get metric data for a single artist over a given time range

import urllib
import datetime as dt
start_dt = dt.datetime(2010,1,1)
end_dt = dt.datetime(2015,7,16)
start = start_dt.strftime("%s")
end = end_dt.strftime("%s")
metric = 'all'

opts = {}
opts['start'] = start
opts['end'] = end
opts['metric'] = metric
data = urllib.urlencode(opts)
print data

artist_id='143'
#url_metricArtist='http://nbsmobile.api3.nextbigsound.com/metrics/artist/'
url_metricArtist='http://swahl.api3.nextbigsound.com/metrics/artist/'
url = url_metricArtist + artist_id + '.json'
#url='http://nbsmobile.api3.nextbigsound.com/metrics/artist/356.json'
print url

reader = codecs.getreader("utf-8")

response = urlopen(url,data)
json_obj = load(response)

start=1262332800&metric=all&end=1437030000
http://swahl.api3.nextbigsound.com/metrics/artist/143.json


In [ ]:
# Get metric data for a single artist over a given time range

import urllib
import datetime as dt
start_dt = dt.datetime(2010,1,1)
end_dt = dt.datetime(2015,7,16)
start = start_dt.strftime("%s")
end = end_dt.strftime("%s")
metric = 'all'

opts = {}
opts['start'] = start
opts['end'] = end
opts['metric'] = metric
data = urllib.urlencode(opts)
print data

artist_id='143'
#url_metricArtist='http://nbsmobile.api3.nextbigsound.com/metrics/artist/'
url_metricArtist='http://swahl.api3.nextbigsound.com/metrics/artist/'
url = url_metricArtist + artist_id + '.json'
#url='http://nbsmobile.api3.nextbigsound.com/metrics/artist/356.json'
print url

In [33]:
reader = codecs.getreader("utf-8")

response = urlopen(url,data)
json_obj = load(response)
url = 'http://api.songkick.com/api/3.0/artists/mbid:507226f4-0842-4f43-950e-81a7017230ec/gigography.json?apikey=CVpeRf49J3duiKg7'
reader = codecs.getreader("utf-8")

response = urlopen(url)
json_obj = load(response)
json_obj['resultsPage'].keys()
event = json_obj['resultsPage']['results']['event'][6]
event.keys()
event['performance'][0].keys()
event['performance'][0]['artist']

{u'displayName': u'Zac Brown Band',
 u'id': 514278,
 u'identifier': [{u'href': u'http://api.songkick.com/api/3.0/artists/mbid:507226f4-0842-4f43-950e-81a7017230ec.json',
   u'mbid': u'507226f4-0842-4f43-950e-81a7017230ec'}],
 u'uri': u'http://www.songkick.com/artists/514278-zac-brown-band?utm_source=35442&utm_medium=partner'}

In [23]:
json_obj[1]['Service']['name']

u'Last.fm'

In [44]:
artists.loc()

356    356
659    659
143    143
Name: id, dtype: object

In [45]:
# readable dates
iservice = 1
imetric = 0

print artists[artists.id == artist_id].name
print json_obj[iservice]['Service']['name']

try:
    mets = json_obj[iservice]['Metric'].keys()
    dates = sorted([int(day)*86400 for day in json_obj[iservice]['Metric'][mets[imetric]].keys()])
    counts = [json_obj[iservice]['Metric'][mets[imetric]][key] 
              for key in json_obj[iservice]['Metric'][mets[imetric]].keys()]
    met = mets[imetric]
    
    print met+':'
    for count,date in zip(counts,dates):
        print dt.datetime.fromtimestamp(
            date
        ).strftime('%Y-%m-%d'), count
except:
    pass

143    Taylor Swift
Name: name, dtype: object
Last.fm
fans:
2015-04-18 1883818
2015-04-22 1884069
2015-04-23 1884310
2015-04-24 1884557
2015-04-25 1882644
2015-04-28 1881594
2015-04-29 1886322
2015-05-01 1883553
2015-05-06 1881874
2015-05-07 1878443
2015-05-08 1867441
2015-05-09 1867137
2015-05-12 1872665
2015-05-13 1872413
2015-05-14 1882396
2015-05-15 1869708
2015-05-16 1869472
2015-05-19 1869194
2015-05-20 1868890
2015-05-21 1885008
2015-05-22 1885554
2015-05-23 1882137
2015-05-26 1877763
2015-05-27 1861067
2015-05-28 1868436
2015-05-29 1868698
2015-05-30 1869197
2015-06-02 1867684
2015-06-03 1867935
2015-06-04 1886020
2015-06-05 1878073
2015-06-06 1878384
2015-06-09 1878655
2015-06-10 1878904
2015-06-11 1875996
2015-06-12 1876387
2015-06-13 1875107
2015-06-16 1875599
2015-06-17 1885380
2015-06-18 1874462
2015-06-19 1873452
2015-06-20 1872910
2015-06-23 1873172
2015-06-24 1878122
2015-06-25 1885711
2015-06-26 1877814
2015-06-27 1877500
2015-06-30 1877138


In [117]:
# including other data in musicbrainz search (discography example)
# /ws/2/artist            area, recording, release, release-group, work

result = musicbrainzngs.get_artist_by_id(artist_id,
              includes=["release-groups"], release_type=["album", "ep"])
for rg in result["artist"]["release-group-list"]:
    print rg['first-release-date'],rg['title'],rg['type']
print result['artist']['release-group-count']
rg.keys()

2003 Hail and Backtrack Compilation
2012-10-12 5 Album Set Compilation
2003 Sail to Montreux Album
 B-Sides Compilation
2013 Oxford Angels: The Rarities Album
2003-05-26 Hail to the Thief Album
 A Total Waste of Time Compilation
2007-10-10 In Rainbows Album
2008-06-02 The Best Of Compilation
2001-01-22 Coup d'État Compilation
2011-02-18 The King of Limbs Album
2009-11 The Best of Radiohead Compilation
1997-05-21 OK Computer Album
2007-12-10 Album Box Set Compilation
1995-03-08 The Bends Album
2001-06-04 Amnesiac Album
1997 Lost Treasures 1993–1997 Compilation
2008 Greatest Hits Compilation
1993-02-22 Pablo Honey Album
 B Sides Compilation
2005 Breaks and Beats Compilation
2004 Towering Above the Rest Compilation
2000-09-27 Kid A Album
2000 Looking Back at... Compilation
1999 World Ballads Collection Compilation
260


['first-release-date',
 'secondary-type-list',
 'primary-type',
 'title',
 'type',
 'id']

In [105]:
result['artist'].keys()

['name',
 'area',
 'gender',
 'life-span',
 'ipi',
 'isni-list',
 'begin-area',
 'ipi-list',
 'sort-name',
 'country',
 'type',
 'id']

In [112]:
# including other data in musicbrainz search (discography example)
# /ws/2/artist            area, recording, release, release-group, work

result2 = musicbrainzngs.get_artist_by_id(artist_id,
              includes=["recordings"])
for rg in result2["artist"]["recording-list"]:
    print rg['title'], rg['length']
print result2['artist']['recording-count']
rg.keys()

"What has been the timetable with this album?" 251000
"Track #8 The Gloaming is next. That's the time just before night falls, isn't it? And this sounds to me like the listener is being eaten by a big technological machine...." 236866
"And what was the mindset in recording? After all, there's a single release!" 48520
"Track #12 is Myxomatosis (a disease that infected the rabbit population of Australia, deliberately, in the 1950s.) Do we detect a Tubeway Army sound here?" 296626
"Track #9 There There is next; and it's a single, unusual indeed for Radiohead...." 223960
"So, the title. How did it come to be chosen?" 344666
"Track #2, Sit Down, Stand Up. Where did that come from?" 278573
"Here's a change (irony!) as a definition of a Radiohead song....Track #4 Backdrifts seems a pretty desolate vision." 157666
"What about touring — has that become enjoyment again?" 68346
"Were there many tracks left over after recording? And can we mention the great song 'Lift', and ask what happened to th

['length', 'id', 'title']

In [94]:
result2['artist'].keys()

['recording-count',
 'recording-list',
 'name',
 'area',
 'country',
 'life-span',
 'isni-list',
 'begin-area',
 'sort-name',
 'type',
 'id']

In [100]:
result3 = musicbrainzngs.get_artist_by_id(artist_id,
              includes=["works"])

In [101]:
result3['artist'].keys()

['work-list',
 'isni-list',
 'name',
 'area',
 'country',
 'life-span',
 'work-count',
 'begin-area',
 'sort-name',
 'type',
 'id']

In [116]:
result4 = musicbrainzngs.get_artist_by_id(artist_id,
              includes=["releases"], release_type=["album", "ep"])
for rg in result4["artist"]["release-list"]:
    print rg['title']
print result4['artist']['release-count']
rg.keys()

Pablo Honey
Pablo Honey
Drill
My Iron Lung
Pablo Honey
Pablo Honey
My Iron Lung
The Bends
Pablo Honey
Pablo Honey
Drill
The Bends (bonus disc: Live EP)
The Bends
Pablo Honey
Pablo Honey
Drill
Drill
1993-09-02: Signal Radio Studios: Cheshire, UK
Pablo Honey
The Bends
Itch
Itch
BBC in Concert 601: Glastonbury Festival 1994
The Bends
My Iron Lung
427


['status',
 'release-event-count',
 'title',
 'country',
 'barcode',
 'release-event-list',
 'text-representation',
 'date',
 'quality',
 'id']

In [103]:
result4['artist'].keys()

['name',
 'area',
 'country',
 'life-span',
 'isni-list',
 'begin-area',
 'release-list',
 'sort-name',
 'release-count',
 'type',
 'id']

In [59]:
# Artist ranking (doesn't exist anymore?)
#type is either nominal/velocity/acceleration
ids = [356, 659, 8309]
atype = "nominal"
resp = json.loads(api.artistRanking(atype, ids))
resp
 

{'message': "Feed doesn't exist", 'status': 'error'}

In [6]:
# services summary
resp = json.loads(api.servicesList())
resp
{key:resp[key]['human'] for key in resp.keys()}

{u'1': u'MySpace',
 u'10': u'SoundCloud',
 u'11': u'Purevolume',
 u'16': u'Vimeo',
 u'17': u'Wikipedia',
 u'2': u'Last.fm',
 u'25': u'Vevo',
 u'33': u'Rdio',
 u'4': u'Facebook',
 u'45': u'Bandcamp',
 u'5': u'Twitter',
 u'52': u'Instagram',
 u'64': u'Tumblr',
 u'66': u'Goodreads (Author)',
 u'68': u'Goodreads (Work)',
 u'7': u'YouTube',
 u'70': u'NBS Score',
 u'77': u'Vine',
 u'8': u'ReverbNation',
 u'85': u'GooglePlus'}

In [48]:
# metrics profile
resp = json.loads(api.metricsProfile("964787"))
resp

{u'fans': {u'16625': 2025423,
  u'16626': 2026836,
  u'16627': 2028263,
  u'16628': 2029624,
  u'16629': 2030809,
  u'16630': 2032132,
  u'16631': 2033389},
 u'favorites': [],
 u'likes': {u'16625': 5149414,
  u'16626': 5153682,
  u'16627': 5158153,
  u'16628': 5162419,
  u'16629': 5166525,
  u'16630': 5170786,
  u'16631': 5175090},
 u'plays': {u'16625': 1364537022,
  u'16626': 1365334694,
  u'16627': 1366186787,
  u'16628': 1366967547,
  u'16629': 1367661611,
  u'16630': 1368391399,
  u'16631': 1369130739},
 u'raters': {u'16625': 5851574,
  u'16626': 5856219,
  u'16627': 5861054,
  u'16628': 5865650,
  u'16629': 5870031,
  u'16630': 5874648,
  u'16631': 5879325}}

In [29]:
# metrics artist
resp = json.loads(api.metricsArtist("356"))
resp

[{u'Metric': {u'fans': [], u'plays': [], u'views': []},
  u'Profile': {u'id': 388, u'url': u'http://www.myspace.com/kanyewest'},
  u'Service': {u'id': 1, u'name': u'MySpace'}},
 {u'Metric': {u'comments': {u'16625': 31268,
    u'16626': 31270,
    u'16628': 31269,
    u'16629': 31271,
    u'16630': 31272,
    u'16631': 31273},
   u'fans': {u'16625': 4023689},
   u'plays': {u'16625': 166542042}},
  u'Profile': {u'id': 174990, u'url': u'http://www.last.fm/music/kanye+west'},
  u'Service': {u'id': 2, u'name': u'Last.fm'}},
 {u'Metric': {u'views': {u'16625': 14979,
    u'16626': 12871,
    u'16627': 10696,
    u'16628': 11233,
    u'16629': 9940,
    u'16630': 12995,
    u'16631': 17440}},
  u'Profile': {u'id': 846577,
   u'url': u'http://en.wikipedia.org/wiki/Kanye_West'},
  u'Service': {u'id': 17, u'name': u'Wikipedia'}},
 {u'Metric': {u'fans': {u'16625': 13179944,
    u'16626': 13198984,
    u'16627': 13216987,
    u'16628': 13233803,
    u'16629': 13251756,
    u'16630': 13269376,
    u

In [38]:
[a['Service']['name'] for a in resp] 

[u'MySpace',
 u'Last.fm',
 u'Wikipedia',
 u'Twitter',
 u'YouTube',
 u'YouTube',
 u'Vevo',
 u'Rdio',
 u'ReverbNation',
 u'NBS Score']

In [40]:
resp[5]

{u'Metric': {u'fans': {u'16625': 2025423,
   u'16626': 2026836,
   u'16627': 2028263,
   u'16628': 2029624,
   u'16629': 2030809,
   u'16630': 2032132,
   u'16631': 2033389},
  u'favorites': [],
  u'likes': {u'16625': 5149414,
   u'16626': 5153682,
   u'16627': 5158153,
   u'16628': 5162419,
   u'16629': 5166525,
   u'16630': 5170786,
   u'16631': 5175090},
  u'plays': {u'16625': 1364537022,
   u'16626': 1365334694,
   u'16627': 1366186787,
   u'16628': 1366967547,
   u'16629': 1367661611,
   u'16630': 1368391399,
   u'16631': 1369130739},
  u'raters': {u'16625': 5851574,
   u'16626': 5856219,
   u'16627': 5861054,
   u'16628': 5865650,
   u'16629': 5870031,
   u'16630': 5874648,
   u'16631': 5879325}},
 u'Profile': {u'id': 964787,
  u'url': u'http://www.youtube.com/channel/UCs6eXM7s8Vl5WcECcRHc2qQ'},
 u'Service': {u'id': 7, u'name': u'YouTube'}}

In [47]:
resp = json.loads(api.profilesArtist("356"))
print resp.keys()
[resp[key]['name'] for key in resp.keys()]

[u'846577', u'1078750', u'174990', u'948411', u'6600125', u'964771', u'4334982', u'964787', u'388', u'964794']


[u'Wikipedia',
 u'Rdio',
 u'Last.fm',
 u'Twitter',
 u'NBS Score',
 u'YouTube',
 u'ReverbNation',
 u'YouTube',
 u'MySpace',
 u'Vevo']

In [2]:
# metrics artist (with options)
#data[start]: a unix timestamp or a text-based date [see php.net/strtotime for details]
#data[end]: a unix timestamp or a text-based date
#data[metric]: all, plays, fans, views, comments, downloads, likes, price
import datetime as dt
start_dt = dt.datetime(2015,1,1)
end_dt = dt.datetime(2015,7,16)
start = start_dt.strftime("%s")
end = end_dt.strftime("%s")
metric = 'all'
resp = api.metricsArtist("356",opt=[start,end,metric])
#resp

http://nbsmobile.api3.nextbigsound.com/metrics/artist/356.json
start=1420099200&metric=all&end=1437030000


In [4]:
resp

'dce0\r\n[\n  {\n    "Service": {\n      "name": "MySpace",\n      "id": 1\n    },\n    "Profile": {\n      "url": "http:\\/\\/www.myspace.com\\/kanyewest",\n      "id": 388\n    },\n    "Metric": {\n      "plays": [\n        \n      ],\n      "fans": [\n        \n      ],\n      "views": [\n        \n      ]\n    }\n  },\n  {\n    "Service": {\n      "name": "Last.fm",\n      "id": 2\n    },\n    "Profile": {\n      "url": "http:\\/\\/www.last.fm\\/music\\/kanye+west",\n      "id": 174990\n    },\n    "Metric": {\n      "plays": {\n        "16542": 163644667,\n        "16548": 164376228,\n        "16549": 164412304,\n        "16550": 164449902,\n        "16551": 164490455,\n        "16554": 164858949,\n        "16555": 164896894,\n        "16557": 164970952,\n        "16562": 164715316,\n        "16563": 164752951,\n        "16564": 164790322,\n        "16565": 164825712,\n        "16568": 165098509,\n        "16569": 165136534,\n        "16570": 165179546,\n        "16571": 165215221

In [78]:
api.Artist("356")

AttributeError: API instance has no attribute 'Artist'

In [29]:
# 



start=1262332800&metric=all&end=1437030000
http://nbsmobile.api3.nextbigsound.com/metrics/artist/356.json


In [32]:
#json_obj

In [33]:
# readable dates
dates = sorted([int(day)*86400 for day in json_obj[1]['Metric']['comments'].keys() ])
for date in dates:
    print dt.datetime.fromtimestamp(
        date
    ).strftime('%Y-%m-%d')

2015-04-16
2015-04-18
2015-04-19
2015-04-20
2015-04-21
2015-04-23
2015-04-25
2015-04-26
2015-04-28
2015-04-29
2015-04-30
2015-05-01
2015-05-02
2015-05-03
2015-05-04
2015-05-06
2015-05-08
2015-05-10
2015-05-11
2015-05-12
2015-05-13
2015-05-14
2015-05-17
2015-05-18
2015-05-19
2015-05-21
2015-05-22
2015-05-23
2015-05-25
2015-05-26
2015-05-30
2015-06-01
2015-06-03
2015-06-04
2015-06-05
2015-06-07
2015-06-08
2015-06-09
2015-06-10
2015-06-15
2015-06-16
2015-06-18
2015-06-19
2015-06-20
2015-06-21
2015-06-22
2015-06-23
2015-06-25
2015-06-26
2015-06-27
2015-06-28
2015-06-29
2015-06-30
2015-07-01
2015-07-02
2015-07-03
2015-07-07
2015-07-08
2015-07-09
2015-07-11
2015-07-12
2015-07-13
2015-07-14


In [61]:
for source in json_obj:
    

{u'Metric': {u'fans': [], u'plays': [], u'views': []},
 u'Profile': {u'id': 388, u'url': u'http://www.myspace.com/kanyewest'},
 u'Service': {u'id': 1, u'name': u'MySpace'}}

In [62]:
json_obj[1]

{u'Metric': {u'comments': {u'16542': 31151,
   u'16544': 31152,
   u'16545': 31153,
   u'16546': 31154,
   u'16547': 31156,
   u'16549': 31160,
   u'16551': 31163,
   u'16552': 31164,
   u'16554': 31169,
   u'16555': 31170,
   u'16556': 31172,
   u'16557': 31174,
   u'16558': 31177,
   u'16559': 31182,
   u'16560': 31186,
   u'16562': 31188,
   u'16564': 31189,
   u'16566': 31190,
   u'16567': 31191,
   u'16568': 31193,
   u'16569': 31192,
   u'16570': 31193,
   u'16573': 31194,
   u'16574': 31195,
   u'16575': 31197,
   u'16577': 31198,
   u'16578': 31199,
   u'16579': 31201,
   u'16581': 31202,
   u'16582': 31203,
   u'16586': 31204,
   u'16588': 31206,
   u'16590': 31212,
   u'16591': 31210,
   u'16592': 31212,
   u'16594': 31213,
   u'16595': 31225,
   u'16596': 31226,
   u'16597': 31227,
   u'16602': 31228,
   u'16603': 31227,
   u'16605': 31229,
   u'16606': 31232,
   u'16607': 31234,
   u'16608': 31236,
   u'16609': 31237,
   u'16610': 31239,
   u'16612': 31241,
   u'16613': 312